In [5]:
import torch
import data_loader
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

In [65]:
from torch.utils.data import TensorDataset, DataLoader

def load_data(N=1000, batch_size=50, seed=42):
    # Load data
    train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)
    train_input = train_input.to(device)
    train_target = train_target.to(device)
    train_classes = train_classes.to(device)
    test_input = test_input.to(device)
    test_target = test_target.to(device)
    test_classes = test_classes.to(device)
    # Normalize data
    mean, std = train_input.mean(), train_input.std()
    train_input.sub_(mean).div_(std)
    test_input.sub_(mean).div_(std)
    
    # Generate dataset
    train_data = TensorDataset(train_input, train_target, train_classes)
    test_data = TensorDataset(test_input, test_target, test_classes)
    
    # For reproducibility
    torch.manual_seed(seed)
    
    # Generate data loader
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size)
    
    return train_loader, test_loader

In [6]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [7]:
# helper.py

# Count the number of parameters
def count_param(model):
    return sum([torch.numel(param) for param in model.parameters()])

In [10]:
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=42)

In [39]:
def compute_nb_errors(model, data_loader):

    nb_data_errors = 0

    for data_input, data_target, data_classes in data_loader:
        data_target = torch.nn.functional.one_hot(data_target)
        output = model(data_input)
        nb_error = torch.sum(torch.argmax(output, dim=1, keepdim=True) != torch.argmax(data_target, dim=1, keepdim=True))
        nb_data_errors += nb_error
        
    return nb_data_errors

In [12]:
def compute_nb_errors_siamese(model, data_loader):

    nb_data_errors = 0
    for data_input, data_target, data_classes in data_loader:
        data_1, data_2 = data_input.unbind(1)               
        output = model(data_1.unsqueeze(1), data_2.unsqueeze(1))
        data_target = torch.nn.functional.one_hot(data_target)
        nb_error = torch.sum(torch.argmax(output, dim=1, keepdim=True) != torch.argmax(data_target, dim=1, keepdim=True))
        nb_data_errors += nb_error
        
    return nb_data_errors

In [13]:
def compute_nb_errors_auxsiamese(model, data_loader):

    nb_data_errors = 0
    for data_input, data_target, data_classes in data_loader:
        data_1, data_2 = data_input.unbind(1)               
        output, aux1, aux2 = model(data_1.unsqueeze(1), data_2.unsqueeze(1))
        data_target = torch.nn.functional.one_hot(data_target)
        nb_error = torch.sum(torch.argmax(output, dim=1, keepdim=True) != torch.argmax(data_target, dim=1, keepdim=True))
        nb_data_errors += nb_error
        
    return nb_data_errors

In [14]:
class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3)    # size [nb, 32, 12, 12]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)   # size [nb, 64, 4, 4]
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(10, 2)
        
    def forward(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 6, 6]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        x = x.view(-1, 256) # size [nb, 256]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [15]:
class SiameseBaseNet(nn.Module):
    def __init__(self):
        super(SiameseBaseNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)    # size [nb, 32, 10, 10]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)   # size [nb, 64, 4, 4]
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(20, 2)
        
    def convs(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 5, 5]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        return x
    
    def forward(self, x1, x2):
        x1 = self.convs(x1)
        x1 = x1.view(-1, 256)
        x1 = F.relu((self.fc1(x1)))
        x1 = F.relu(self.fc2(x1))
        
        x2 = self.convs(x2)
        x2 = x2.view(-1, 256)
        x2 = F.relu(self.fc1(x2))
        x2 = F.relu(self.fc2(x2))
        
        x = torch.cat([x1, x2], dim=1)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        x = torch.sigmoid(self.fc3(x))
        
        return x

In [16]:
class AuxsiameseBaseNet(nn.Module):
    def __init__(self):
        super(AuxsiameseBaseNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)    # size [nb, 32, 10, 10]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)   # size [nb, 64, 4, 4]
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(20, 2)
        
    def convs(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 5, 5]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        return x
    
    def forward(self, x1, x2):
        x1 = self.convs(x1)
        x1 = x1.view(-1, 256)
        x1 = F.relu((self.fc1(x1)))
        x1 = F.relu(self.fc2(x1))
        
        x2 = self.convs(x2)
        x2 = x2.view(-1, 256)
        x2 = F.relu(self.fc1(x2))
        x2 = F.relu(self.fc2(x2))
        
        x = torch.cat([x1, x2], dim=1)
        aux1 = F.softmax(x1)
        aux2 = F.softmax(x2)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        x = torch.sigmoid(self.fc3(x))
        
        return x, aux1, aux2

In [17]:
class AuxBaseNet(nn.Module):
    def __init__(self):
        super(AuxBaseNet, self).__init__()
        self.conv11 = nn.Conv2d(1, 32, kernel_size=3)    # size [nb, 32, 10, 10]
        self.conv21 = nn.Conv2d(32, 64, kernel_size=3)   # size [nb, 64, 4, 4]
        self.fc11 = nn.Linear(256, 200)
        self.fc21 = nn.Linear(200, 10)
        self.conv12 = nn.Conv2d(1, 32, kernel_size=5)    # size [nb, 32, 10, 10]
        self.conv22 = nn.Conv2d(32, 64, kernel_size=2)   # size [nb, 64, 4, 4]
        self.fc12 = nn.Linear(256, 200)
        self.fc22 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(20, 2)
        
    def convs(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 5, 5]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        return x
    
    def forward(self, x1, x2):
        x1 = F.relu(F.max_pool2d(self.conv11(x1), kernel_size=2)) # size [nb, 32, 5, 5]  
        x1 = F.relu(F.max_pool2d(self.conv21(x1), kernel_size=2)) # size [nb, 64, 2, 2]
        x1 = x1.view(-1, 256)
        x1 = F.relu((self.fc11(x1)))
        x1 = F.relu(self.fc21(x1))
        
        x2 = F.relu(F.max_pool2d(self.conv12(x2), kernel_size=2)) # size [nb, 32, 5, 5]  
        x2 = F.relu(F.max_pool2d(self.conv22(x2), kernel_size=2)) # size [nb, 64, 2, 2]
        x2 = x2.view(-1, 256)
        x2 = F.relu((self.fc12(x2)))
        x2 = F.relu(self.fc22(x2))
        
        x = torch.cat([x1, x2], dim=1)
        aux1 = F.softmax(x1)
        aux2 = F.softmax(x2)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        x = torch.sigmoid(self.fc3(x))
        
        return x, aux1, aux2

In [18]:
# ResNetBlock with skip-connection and batch normalization
class ResNetBlock(nn.Module):
    def __init__(self, nb_channels, kernel_size, dropout = 0):
        super().__init__()

        self.conv1 = nn.Conv2d(nb_channels, nb_channels,
                               kernel_size = kernel_size,
                               padding = (kernel_size - 1) // 2)

        self.bn1 = nn.BatchNorm2d(nb_channels)

        self.conv2 = nn.Conv2d(nb_channels, nb_channels,
                               kernel_size = kernel_size,
                               padding = (kernel_size - 1) // 2)

        self.bn2 = nn.BatchNorm2d(nb_channels)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        y = self.conv1(x)
        y = self.bn1(y)
        y = self.dropout(y)
        y = F.relu(y)
        y = self.conv2(y)
        y = self.bn2(y)
        y = self.dropout(y)
        y = y + x
        y = F.relu(y)

        return y

In [19]:
class ResNet(nn.Module):

    def __init__(self, nb_residual_blocks, input_channels, nb_channels,
                 kernel_size = 3, nb_classes = 10, dropout = 0):
        super().__init__()

        self.conv = nn.Conv2d(input_channels, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.bn = nn.BatchNorm2d(nb_channels)

        self.resnet_blocks = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size, dropout)
              for _ in range(nb_residual_blocks))
        )

        self.fc = nn.Linear(288, nb_classes)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x = self.resnet_blocks(x)
        x = F.avg_pool2d(x, 4).view(x.size(0), -1)
        x = torch.sigmoid(self.fc(x))
        return x

In [20]:
class SiameseResNet(nn.Module):

    def __init__(self, nb_residual_blocks, input_channels, nb_channels,
                 kernel_size = 3, nb_classes = 10, dropout = 0):
        super().__init__()

        self.conv = nn.Conv2d(input_channels, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.bn = nn.BatchNorm2d(nb_channels)

        self.resnet_blocks = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size, dropout)
              for _ in range(nb_residual_blocks))
        )

        self.fc = nn.Linear(576, nb_classes)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x1, x2):
        x1 = F.relu(self.bn(self.conv(x1)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x1 = self.resnet_blocks(x1)
        x1 = F.avg_pool2d(x1, 4).view(x1.size(0), -1)
        
        x2 = F.relu(self.bn(self.conv(x2)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x2 = self.resnet_blocks(x2)
        x2 = F.avg_pool2d(x2, 4).view(x2.size(0), -1)
        
        x = torch.cat([x1, x2], dim=1)
        #aux1 = F.softmax(self.fc1(x1)
        #aux2 = F.softmax(self.fc1(x2)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))      
        x = torch.sigmoid(self.fc(x))
        
        return x

In [82]:
class AuxResNet(nn.Module):

    def __init__(self, nb_residual_blocks, input_channels, nb_channels,
                 kernel_size = 3, nb_classes = 10, dropout = 0):
        super().__init__()

        self.conv1 = nn.Conv2d(input_channels, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.conv2 = nn.Conv2d(input_channels, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.bn1 = nn.BatchNorm2d(nb_channels)
        self.bn2 = nn.BatchNorm2d(nb_channels)

        self.resnet_blocks1 = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size, dropout)
              for _ in range(nb_residual_blocks))
        )
        self.resnet_blocks2 = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size, dropout)
              for _ in range(nb_residual_blocks))
        )
        
        self.fc = nn.Linear(576, nb_classes)
        self.fc1 = nn.Linear(288, 10)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x1, x2):
        x1 = F.relu(self.bn1(self.conv1(x1)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x1 = self.resnet_blocks1(x1)
        x1 = F.avg_pool2d(x1, 4).view(x1.size(0), -1)
        
        x2 = F.relu(self.bn2(self.conv2(x2)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x2 = self.resnet_blocks2(x2)
        x2 = F.avg_pool2d(x2, 4).view(x2.size(0), -1)
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.sigmoid(self.fc(x))
        aux1 = F.softmax(self.fc1(x1))
        aux2 = F.softmax(self.fc1(x2))
        #aux1 = F.softmax(x1)
        #aux2 = F.softmax(x2)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))      
        
        return x, aux1, aux2

In [75]:
class AuxsiameseResNet(nn.Module):

    def __init__(self, nb_residual_blocks, input_channels, nb_channels,
                 kernel_size = 3, nb_classes = 10, dropout = 0):
        super().__init__()

        self.conv = nn.Conv2d(input_channels, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.bn = nn.BatchNorm2d(nb_channels)

        self.resnet_blocks = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size, dropout)
              for _ in range(nb_residual_blocks))
        )

        self.fc = nn.Linear(576, nb_classes)
        self.fc1 = nn.Linear(288, 10)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x1, x2):
        x1 = F.relu(self.bn(self.conv(x1)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x1 = self.resnet_blocks(x1)
        x1 = F.avg_pool2d(x1, 4).view(x1.size(0), -1)
        
        x2 = F.relu(self.bn(self.conv(x2)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x2 = self.resnet_blocks(x2)
        x2 = F.avg_pool2d(x2, 4).view(x2.size(0), -1)
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.sigmoid(self.fc(x))
        aux1 = F.softmax(self.fc1(x1))
        aux2 = F.softmax(self.fc1(x2))
        #aux1 = F.softmax(x1)
        #aux2 = F.softmax(x2)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))      
        
        return x, aux1, aux2

In [22]:
model = AuxsiameseResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)

# Calculate the number of parameters in the model
count_param(model)

190668

In [23]:
model = SiameseResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)

# Calculate the number of parameters in the model
count_param(model)

187778

In [83]:
model = AuxResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)

# Calculate the number of parameters in the model
count_param(model)

377292

In [24]:
model = ResNet(nb_residual_blocks = 10, input_channels = 2, nb_channels = 32, kernel_size = 3, nb_classes = 2)
# Calculate the number of parameters in the model
count_param(model)

187490

In [25]:
model = BaseNet()

# Calculate the number of parameters in the model
count_param(model)

72536

In [26]:
model = SiameseBaseNet()

# Calculate the number of parameters in the model
count_param(model)

72268

In [27]:
model = AuxsiameseBaseNet()

# Calculate the number of parameters in the model
count_param(model)

72268

In [28]:
model = AuxBaseNet()

# Calculate the number of parameters in the model
count_param(model)

134766

In [66]:
def train(model, train_loader, eta, decay, n_epochs=25, verbose=False, siamese=False, aux=False, alpha = 0):

    #binary_crit = nn.CrossEntropyLoss()
    binary_crit = torch.nn.BCELoss()
    aux_crit = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=eta, weight_decay=decay)
    
    tr_losses = []
    tr_accuracies = []

    for e in range(n_epochs):
        # Reset training/validation loss
        tr_loss = 0

        # Training model
        model.train()

        for train_input, train_target, train_classes in iter(train_loader):
            train_target = torch.nn.functional.one_hot(train_target)
            # Forward pass
            
            if siamese == True:
                train_1, train_2 = train_input.unbind(1)
                if aux == True:
                    output, aux1, aux2 = model(train_1.unsqueeze(1), train_2.unsqueeze(1))
                else:
                    output = model(train_1.unsqueeze(1), train_2.unsqueeze(1))
            elif aux == True:
                train_1, train_2 = train_input.unbind(1)
                output, aux1, aux2 = model(train_1.unsqueeze(1), train_2.unsqueeze(1))
            else:
                output = model(train_input)
                
            # Binary classification loss
            binary_loss = binary_crit(output, train_target.float())
            total_loss = binary_loss
            
            # Auxiliary loss
            if aux == True:

                aux_loss1 = aux_crit(aux1, train_classes[:,0])
                aux_loss2 = aux_crit(aux2, train_classes[:,1])
                aux_loss = aux_loss1 + aux_loss2
                total_loss = binary_loss + aux_loss * alpha
        
            # Total loss = Binary loss + aux loss * alpha
            
            tr_loss += total_loss

            # Backward pass
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

        # Collect accuracy data
        # tr_accuracies.append(compute_nb_errors_siamese(model, train_loader)/1000)

        # Collect loss data
        tr_losses.append(tr_loss)

        if verbose:
            print('Epoch %d/%d, Binary loss: %.3f' %
                  (e+1, n_epochs, tr_loss))

In [67]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
model = AuxsiameseResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
model.to(device)
#model = BaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=True, aux=True, alpha = 0.1)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1/25, Binary loss: 22.082
Epoch 2/25, Binary loss: 16.807
Epoch 3/25, Binary loss: 14.288
Epoch 4/25, Binary loss: 12.621
Epoch 5/25, Binary loss: 11.836
Epoch 6/25, Binary loss: 10.218
Epoch 7/25, Binary loss: 9.515
Epoch 8/25, Binary loss: 8.744
Epoch 9/25, Binary loss: 8.608
Epoch 10/25, Binary loss: 8.244
Epoch 11/25, Binary loss: 8.606
Epoch 12/25, Binary loss: 8.923
Epoch 13/25, Binary loss: 8.164
Epoch 14/25, Binary loss: 7.558
Epoch 15/25, Binary loss: 7.285
Epoch 16/25, Binary loss: 7.127
Epoch 17/25, Binary loss: 7.014
Epoch 18/25, Binary loss: 6.707
Epoch 19/25, Binary loss: 6.664
Epoch 20/25, Binary loss: 6.642
Epoch 21/25, Binary loss: 6.631
Epoch 22/25, Binary loss: 6.623
Epoch 23/25, Binary loss: 6.615
Epoch 24/25, Binary loss: 6.610
Epoch 25/25, Binary loss: 6.608
Spend 1.216382e+02 s
tensor(1., device='cuda:0') tensor(0.8770, device='cuda:0')


In [84]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#model = SiameseBaseNet()
#model = BaseNet()
model = AuxsiameseResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
model.to(device
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=False, aux=True, alpha = 0.3)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1/25, Binary loss: 40.103
Epoch 2/25, Binary loss: 31.625
Epoch 3/25, Binary loss: 28.229
Epoch 4/25, Binary loss: 26.540
Epoch 5/25, Binary loss: 24.686
Epoch 6/25, Binary loss: 23.148
Epoch 7/25, Binary loss: 22.162
Epoch 8/25, Binary loss: 21.673
Epoch 9/25, Binary loss: 20.688
Epoch 10/25, Binary loss: 19.981
Epoch 11/25, Binary loss: 19.738
Epoch 12/25, Binary loss: 19.896
Epoch 13/25, Binary loss: 19.436
Epoch 14/25, Binary loss: 19.325
Epoch 15/25, Binary loss: 19.050
Epoch 16/25, Binary loss: 18.952
Epoch 17/25, Binary loss: 18.856
Epoch 18/25, Binary loss: 18.774
Epoch 19/25, Binary loss: 18.752
Epoch 20/25, Binary loss: 17.955
Epoch 21/25, Binary loss: 17.716
Epoch 22/25, Binary loss: 17.694
Epoch 23/25, Binary loss: 17.623
Epoch 24/25, Binary loss: 17.593
Epoch 25/25, Binary loss: 17.584
Spend 1.226981e+02 s
tensor(1., device='cuda:0') tensor(0.8610, device='cuda:0')


In [1081]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
model = SiameseResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
#model = BaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=True)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_siamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Epoch 1/25, Binary loss: 11.641
Epoch 2/25, Binary loss: 8.032
Epoch 3/25, Binary loss: 5.910
Epoch 4/25, Binary loss: 5.168
Epoch 5/25, Binary loss: 4.474
Epoch 6/25, Binary loss: 3.215
Epoch 7/25, Binary loss: 3.069
Epoch 8/25, Binary loss: 3.053
Epoch 9/25, Binary loss: 2.638
Epoch 10/25, Binary loss: 2.044
Epoch 11/25, Binary loss: 2.564
Epoch 12/25, Binary loss: 1.829
Epoch 13/25, Binary loss: 1.344
Epoch 14/25, Binary loss: 0.564
Epoch 15/25, Binary loss: 0.404
Epoch 16/25, Binary loss: 0.158
Epoch 17/25, Binary loss: 0.359
Epoch 18/25, Binary loss: 0.458
Epoch 19/25, Binary loss: 0.738
Epoch 20/25, Binary loss: 0.517
Epoch 21/25, Binary loss: 0.761
Epoch 22/25, Binary loss: 0.393
Epoch 23/25, Binary loss: 0.200
Epoch 24/25, Binary loss: 0.130
Epoch 25/25, Binary loss: 0.067
Spend 4.333027e+02 s
tensor(0.9990) tensor(0.8690)


In [1055]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#model = SiameseBaseNet()
model = ResNet(nb_residual_blocks = 3, input_channels = 2, nb_channels = 32, kernel_size = 3, nb_classes = 2, dropout = 0.1)
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=False)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Epoch 1/25, Binary loss: 12.579
Epoch 2/25, Binary loss: 10.303
Epoch 3/25, Binary loss: 9.214
Epoch 4/25, Binary loss: 7.878
Epoch 5/25, Binary loss: 7.113
Epoch 6/25, Binary loss: 6.340
Epoch 7/25, Binary loss: 5.647
Epoch 8/25, Binary loss: 5.246
Epoch 9/25, Binary loss: 4.497
Epoch 10/25, Binary loss: 4.413
Epoch 11/25, Binary loss: 4.452
Epoch 12/25, Binary loss: 3.421
Epoch 13/25, Binary loss: 2.968
Epoch 14/25, Binary loss: 3.099
Epoch 15/25, Binary loss: 2.219
Epoch 16/25, Binary loss: 1.569
Epoch 17/25, Binary loss: 1.443
Epoch 18/25, Binary loss: 1.220
Epoch 19/25, Binary loss: 0.891
Epoch 20/25, Binary loss: 1.047
Epoch 21/25, Binary loss: 0.979
Epoch 22/25, Binary loss: 0.917
Epoch 23/25, Binary loss: 1.077
Epoch 24/25, Binary loss: 0.904
Epoch 25/25, Binary loss: 0.614
Spend 6.177664e+01 s
tensor(0.9980) tensor(0.8360)


In [878]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
model = SiameseBaseNet()
#model = BaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=True)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_siamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Epoch 1/25, Binary loss: 2.743
Epoch 2/25, Binary loss: 2.704
Epoch 3/25, Binary loss: 2.636
Epoch 4/25, Binary loss: 2.529
Epoch 5/25, Binary loss: 2.372
Epoch 6/25, Binary loss: 2.187
Epoch 7/25, Binary loss: 1.991
Epoch 8/25, Binary loss: 1.832
Epoch 9/25, Binary loss: 1.687
Epoch 10/25, Binary loss: 1.552
Epoch 11/25, Binary loss: 1.421
Epoch 12/25, Binary loss: 1.303
Epoch 13/25, Binary loss: 1.189
Epoch 14/25, Binary loss: 1.096
Epoch 15/25, Binary loss: 1.001
Epoch 16/25, Binary loss: 0.925
Epoch 17/25, Binary loss: 0.836
Epoch 18/25, Binary loss: 0.760
Epoch 19/25, Binary loss: 0.696
Epoch 20/25, Binary loss: 0.614
Epoch 21/25, Binary loss: 0.548
Epoch 22/25, Binary loss: 0.487
Epoch 23/25, Binary loss: 0.422
Epoch 24/25, Binary loss: 0.361
Epoch 25/25, Binary loss: 0.307
Spend 9.664215e+00 s
tensor(0.9890) tensor(0.8580)


In [883]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#modeF.avg_pool2dl = SiameseBaseNet()
model = BaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=False)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Epoch 1/25, Binary loss: 2.775
Epoch 2/25, Binary loss: 2.737
Epoch 3/25, Binary loss: 2.680
Epoch 4/25, Binary loss: 2.574
Epoch 5/25, Binary loss: 2.400
Epoch 6/25, Binary loss: 2.175
Epoch 7/25, Binary loss: 1.978
Epoch 8/25, Binary loss: 1.849
Epoch 9/25, Binary loss: 1.777
Epoch 10/25, Binary loss: 1.617
Epoch 11/25, Binary loss: 1.531
Epoch 12/25, Binary loss: 1.432
Epoch 13/25, Binary loss: 1.335
Epoch 14/25, Binary loss: 1.232
Epoch 15/25, Binary loss: 1.146
Epoch 16/25, Binary loss: 1.058
Epoch 17/25, Binary loss: 0.955
Epoch 18/25, Binary loss: 0.865
Epoch 19/25, Binary loss: 0.781
Epoch 20/25, Binary loss: 0.770
Epoch 21/25, Binary loss: 0.651
Epoch 22/25, Binary loss: 0.549
Epoch 23/25, Binary loss: 0.616
Epoch 24/25, Binary loss: 0.616
Epoch 25/25, Binary loss: 0.484
Spend 4.811039e+00 s
tensor(0.9780) tensor(0.8450)


In [687]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#model = SiameseBaseNet()
#model = BaseNet()
model = AuxsiameseBaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=True, aux=True, alpha = 0.3)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

<ipython-input-673-21efc383ef87>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux1 = F.softmax(x1)
<ipython-input-673-21efc383ef87>:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux2 = F.softmax(x2)


Epoch 1/25, Binary loss: 41.301
Epoch 2/25, Binary loss: 40.527
Epoch 3/25, Binary loss: 39.194
Epoch 4/25, Binary loss: 37.589
Epoch 5/25, Binary loss: 35.413
Epoch 6/25, Binary loss: 33.461
Epoch 7/25, Binary loss: 32.224
Epoch 8/25, Binary loss: 31.236
Epoch 9/25, Binary loss: 30.435
Epoch 10/25, Binary loss: 29.611
Epoch 11/25, Binary loss: 29.025
Epoch 12/25, Binary loss: 28.527
Epoch 13/25, Binary loss: 28.140
Epoch 14/25, Binary loss: 27.689
Epoch 15/25, Binary loss: 27.367
Epoch 16/25, Binary loss: 27.219
Epoch 17/25, Binary loss: 27.102
Epoch 18/25, Binary loss: 26.959
Epoch 19/25, Binary loss: 26.890
Epoch 20/25, Binary loss: 26.829
Epoch 21/25, Binary loss: 26.785
Epoch 22/25, Binary loss: 26.759
Epoch 23/25, Binary loss: 26.788
Epoch 24/25, Binary loss: 26.739
Epoch 25/25, Binary loss: 26.726
Spend 1.036312e+01 s
tensor(1.) tensor(0.8700)


In [712]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#model = SiameseBaseNet()
#model = BaseNet()
model = AuxBaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=False, aux=True, alpha = 0.3)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

<ipython-input-699-f890ddbc2f26>:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux1 = F.softmax(x1)
<ipython-input-699-f890ddbc2f26>:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux2 = F.softmax(x2)


Epoch 1/25, Binary loss: 41.354
Epoch 2/25, Binary loss: 40.233
Epoch 3/25, Binary loss: 38.545
Epoch 4/25, Binary loss: 36.119
Epoch 5/25, Binary loss: 34.638
Epoch 6/25, Binary loss: 32.981
Epoch 7/25, Binary loss: 32.049
Epoch 8/25, Binary loss: 31.040
Epoch 9/25, Binary loss: 30.221
Epoch 10/25, Binary loss: 29.890
Epoch 11/25, Binary loss: 29.816
Epoch 12/25, Binary loss: 28.891
Epoch 13/25, Binary loss: 28.312
Epoch 14/25, Binary loss: 27.903
Epoch 15/25, Binary loss: 27.736
Epoch 16/25, Binary loss: 27.181
Epoch 17/25, Binary loss: 26.733
Epoch 18/25, Binary loss: 26.162
Epoch 19/25, Binary loss: 26.032
Epoch 20/25, Binary loss: 25.755
Epoch 21/25, Binary loss: 25.475
Epoch 22/25, Binary loss: 25.369
Epoch 23/25, Binary loss: 25.333
Epoch 24/25, Binary loss: 25.143
Epoch 25/25, Binary loss: 25.044
Spend 9.934680e+00 s
tensor(1.) tensor(0.8390)


In [78]:
accuracies = []
times = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=100, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    model = BaseNet()
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False)
    time2 = time.perf_counter()
    times.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies.append(te_accuracy)

print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies).mean(), torch.tensor(accuracies).std()))

tensor(1., device='cuda:0') tensor(0.8350, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8010, device='cuda:0')
tensor(0.9830, device='cuda:0') tensor(0.8370, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8430, device='cuda:0')
tensor(0.9810, device='cuda:0') tensor(0.8170, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8380, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8100, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8280, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8480, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8280, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8330, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8200, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8200, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8500, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8220, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.7860, device='cuda:0')
tensor(1., devic

In [90]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times).mean(), torch.tensor(times).std()))

Mean: 6.692, Std: 0.067


In [79]:
accuracies1 = []
times1 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    model = SiameseBaseNet()
    model.to(device)
    #model = BaseNet()
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True)
    time2 = time.perf_counter()
    times1.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_siamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies1.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies1).mean(), torch.tensor(accuracies1).std()))

tensor(1., device='cuda:0') tensor(0.8480, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8780, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8390, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8690, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8710, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8500, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8570, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8400, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8580, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8500, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8560, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8370, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8460, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8580, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8500, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8550, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.847

In [91]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times1).mean(), torch.tensor(times1).std()))

Mean: 16.312, Std: 0.113


In [80]:
accuracies2 = []
times2 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxBaseNet()
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False, aux=True, alpha = 0.6)
    time2 = time.perf_counter()
    times2.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies2.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies2).mean(), torch.tensor(accuracies2).std()))

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.9970, device='cuda:0') tensor(0.8480, device='cuda:0')
tensor(0.9880, device='cuda:0') tensor(0.8390, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8480, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8260, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8500, device='cuda:0')
tensor(0.9980, device='cuda:0') tensor(0.8460, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8230, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8460, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8570, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8210, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8500, device='cuda:0')
tensor(0.9980, device='cuda:0') tensor(0.8550, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8470, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8350, device='cuda:0')
tensor(0.9910, device='cuda:0') tensor(0.7990, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8320, device='cuda:0')
tensor(0.9950, d

In [92]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times2).mean(), torch.tensor(times2).std()))

Mean: 22.799, Std: 0.173


In [87]:
accuracies3 = []
times3 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxsiameseBaseNet()
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True, aux=True, alpha = 0.5)
    time2 = time.perf_counter()
    times3.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies3.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies3).mean(), torch.tensor(accuracies3).std()))

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(1., device='cuda:0') tensor(0.8650, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8800, device='cuda:0')
tensor(0.9950, device='cuda:0') tensor(0.8510, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8760, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8690, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8470, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8650, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8510, device='cuda:0')
tensor(0.9970, device='cuda:0') tensor(0.8610, device='cuda:0')
tensor(0.9930, device='cuda:0') tensor(0.8480, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8580, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8460, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8600, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8650, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8630, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8680, device='cuda:0')
tensor(1., device='cuda:0') 

In [93]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times3).mean(), torch.tensor(times3).std()))

Mean: 18.461, Std: 0.171


In [72]:
accuracies4 = []
times4 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = ResNet(nb_residual_blocks = 10, input_channels = 2, nb_channels = 32, kernel_size = 3, nb_classes = 2)
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False)
    time2 = time.perf_counter()
    times4.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies4.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies4).mean(), torch.tensor(accuracies4).std()))

tensor(1., device='cuda:0') tensor(0.8180, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8190, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8430, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8010, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8510, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8360, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8380, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8430, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8370, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8590, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8180, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8510, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8470, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8240, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8530, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8280, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.854

In [94]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times4).mean(), torch.tensor(times4).std()))

Mean: 84.119, Std: 0.164


In [73]:
accuracies5 = []
times5 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    model = SiameseResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
    #model = BaseNet()
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True)
    time2 = time.perf_counter()
    times5.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_siamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies5.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies5).mean(), torch.tensor(accuracies5).std()))

tensor(1., device='cuda:0') tensor(0.8670, device='cuda:0')
tensor(0.9970, device='cuda:0') tensor(0.8650, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8830, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8730, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8820, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8740, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8700, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8720, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8770, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8610, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8680, device='cuda:0')
tensor(0.9880, device='cuda:0') tensor(0.8370, device='cuda:0')
tensor(0.9950, device='cuda:0') tensor(0.8470, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8640, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8520, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8800, device='cuda:0')
tensor(1., device='cuda:0') 

In [95]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times5).mean(), torch.tensor(times5).std()))

Mean: 118.672, Std: 0.185


In [89]:
accuracies6 = []
times6 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False, aux=True, alpha = 0.9)
    time2 = time.perf_counter()
    times6.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies6.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies6).mean(), torch.tensor(accuracies6).std()))

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.9870, device='cuda:0') tensor(0.8550, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8690, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8760, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8690, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8790, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8800, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8590, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.9010, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8720, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8780, device='cuda:0')
tensor(0.9950, device='cuda:0') tensor(0.8660, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8760, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8860, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8740, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8920, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8590, device='cuda:0')
tensor(1., device='cuda:

In [96]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times6).mean(), torch.tensor(times6).std()))

Mean: 164.991, Std: 0.467


In [85]:
accuracies7 = []
times7 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxsiameseResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True, aux=True, alpha = 0.7)
    time2 = time.perf_counter()
    times7.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies7.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies7).mean(), torch.tensor(accuracies7).std()))

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(1., device='cuda:0') tensor(0.9170, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8890, device='cuda:0')
tensor(0.9830, device='cuda:0') tensor(0.8790, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.9060, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8930, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8900, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.9130, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8820, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8970, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.9110, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8780, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8870, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8680, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.9050, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8860, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8840, device='cuda:0')
tensor(0.9970, device='cuda:0') tens

In [97]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times7).mean(), torch.tensor(times7).std()))

Mean: 122.711, Std: 0.480


In [419]:
gammas = torch.logspace(start=-4, end=-2, steps=5)
decays = torch.logspace(start=-13, end=-8, steps=6)
accuracies = torch.empty((len(gammas), len(decays)))
model = BaseNet()
for j in range(len(gammas)):
    for k in range(len(decays)):
        accurate = []
        for i in range(10):
            train_loader, test_loader = load_data(N=1000, batch_size=50, seed=42)
            train(model, train_loader, gammas[j], decays[k], 25, verbose=False)
            te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
            accurate.append(te_accuracy)
        accuracies[j,k] = torch.Tensor(accurate).mean()

KeyboardInterrupt: 

In [754]:
for j in range(10):
    accuracies3 = []
    times3 = []

    for i in range(10):
        train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
        time1 = time.perf_counter()
        #model = SiameseBaseNet()
        #model = BaseNet()
        model = AuxBaseNet()
        train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False, aux=True, alpha = j/10)
        time2 = time.perf_counter()
        times3.append(time2 - time1)

        tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies3.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies3).mean(), torch.tensor(accuracies3).std()))

<ipython-input-744-d08fd1651f1c>:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux1 = F.softmax(x1)
<ipython-input-744-d08fd1651f1c>:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux2 = F.softmax(x2)


Mean: 0.833, Std: 0.009
Mean: 0.831, Std: 0.008
Mean: 0.819, Std: 0.047
Mean: 0.834, Std: 0.008
Mean: 0.830, Std: 0.013
Mean: 0.833, Std: 0.010
Mean: 0.835, Std: 0.018
Mean: 0.833, Std: 0.014
Mean: 0.833, Std: 0.014
Mean: 0.833, Std: 0.017


In [755]:
for j in range(10):
    accuracies2 = []
    times2 = []

    for i in range(10):
        train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
        time1 = time.perf_counter()
        #model = SiameseBaseNet()
        #model = BaseNet()
        model = AuxsiameseBaseNet()
        train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True, aux=True, alpha = j/10)
        time2 = time.perf_counter()
        times2.append(time2 - time1)

        tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies2.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies2).mean(), torch.tensor(accuracies2).std())) 

<ipython-input-743-f9afae9d6bc5>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux1 = F.softmax(x1)
<ipython-input-743-f9afae9d6bc5>:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux2 = F.softmax(x2)


Mean: 0.860, Std: 0.015
Mean: 0.859, Std: 0.011
Mean: 0.858, Std: 0.012
Mean: 0.857, Std: 0.011
Mean: 0.861, Std: 0.010
Mean: 0.864, Std: 0.011
Mean: 0.859, Std: 0.011
Mean: 0.857, Std: 0.017
Mean: 0.859, Std: 0.018
Mean: 0.859, Std: 0.022


In [70]:
for j in range(11):
    accuracies999 = []
    times999 = []

    for i in range(10):
        number = torch.randint(1,50,(1,))
        train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
        time1 = time.perf_counter()
        model = AuxsiameseResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
        model.to(device)
        #model = BaseNet()
        train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True, aux=True, alpha = j/10)
        time2 = time.perf_counter()

        tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies999.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies999).mean(), torch.tensor(accuracies999).std())) 

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Mean: 0.871, Std: 0.009
Mean: 0.885, Std: 0.003
Mean: 0.889, Std: 0.004
Mean: 0.892, Std: 0.006
Mean: 0.889, Std: 0.014
Mean: 0.893, Std: 0.006
Mean: 0.892, Std: 0.003
Mean: 0.896, Std: 0.003
Mean: 0.891, Std: 0.007
Mean: 0.890, Std: 0.012
Mean: 0.890, Std: 0.018


In [86]:
for j in range(11):
    accuracies9999 = []
    times9999 = []

    for i in range(10):
        number = torch.randint(1,50,(1,))
        train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
        time1 = time.perf_counter()
        model = AuxResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
        model.to(device)
        #model = BaseNet()
        train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False, aux=True, alpha = j/10)
        time2 = time.perf_counter()
        times999.append(time2 - time1)
        
        tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies9999.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies9999).mean(), torch.tensor(accuracies999).std())) 

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Mean: 0.843, Std: 0.018
Mean: 0.858, Std: 0.018
Mean: 0.859, Std: 0.018
Mean: 0.861, Std: 0.018
Mean: 0.860, Std: 0.018
Mean: 0.857, Std: 0.018
Mean: 0.854, Std: 0.018
Mean: 0.854, Std: 0.018
Mean: 0.860, Std: 0.018
Mean: 0.868, Std: 0.018
Mean: 0.867, Std: 0.018
